# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix (q=0,ω=0) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.234782188859                   -0.50    8.0
  2   -7.250316972313       -1.81       -1.40    1.0
  3   -7.251319890071       -3.00       -2.15    4.0
  4   -7.251318105555   +   -5.75       -2.42    4.0
  5   -7.251336754679       -4.73       -2.95    3.0
  6   -7.251337953192       -5.92       -3.20    2.0
  7   -7.251338771399       -6.09       -3.98    1.0
  8   -7.251338794198       -7.64       -4.24    3.0
  9   -7.251338798164       -8.40       -4.73    2.0
 10   -7.251338798675       -9.29       -5.33    2.0
 11   -7.251338798699      -10.62       -5.69    3.0
 12   -7.251338798704      -11.25       -6.52    2.0
 13   -7.251338798705      -12.99       -6.90    4.0
 14   -7.251338798705      -13.85       -7.29    3.0
 15   -7.251338798705      -14.45       -7.87    3.0


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Apply ε† = 1 - χ0 (vc + fxc)

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end

eps_fun (generic function with 1 method)

eager diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.03053003426782281
[ Info: Arnoldi iteration step 2: normres = 0.7579310174027554
[ Info: Arnoldi iteration step 3: normres = 0.7334720764169167
[ Info: Arnoldi iteration step 4: normres = 0.48136247956444594
[ Info: Arnoldi iteration step 5: normres = 0.2654945076490109
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (3.41e-02, 7.46e-02, 1.54e-01, 1.87e-01, 7.14e-02)
[ Info: Arnoldi iteration step 6: normres = 0.41479628150257086
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (1.19e-02, 1.24e-01, 3.84e-01, 6.67e-02, 1.48e-02)
[ Info: Arnoldi iteration step 7: normres = 0.10072538190090473
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (6.37e-04, 2.06e-02, 3.04e-02, 2.83e-02, 8.58e-02)
[ Info: Arnoldi iteration step 8: normres = 0.1151040635279291
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (3.15e-